# REPORT CONFIGURATION
This section should be populated with the following values:
* Query - The JIRA JQL query to identify the relevant scope
* Title - A description of the scope to include in the reports.
* Days to Extrapolate - For forecasting completion & creation rates, how much historically data should be considered.  Generally, data should be considered in 7-day blocks to avoid any weekend bias.
* Start Dates - The first date to include in reports.
* End Dates - The last date to include in reports.  This date can be future dated.  All dates beyond the current date will be based on extrapolation.

In [ ]:
jira_reconnect = False # You will only be prompted to connect the first time.  If credentials are entered incorrectly, you can change this to true to reconnect.

# Examples:
# query = 'issuekey in portfolioChildIssuesOf("FSP-8899")'
# title = "Dev Distractions Scope"
# days_to_extrapolate = 14
# start_date="2024-07-01"
# end_date="2024-10-01"

query = 'issuekey in portfolioChildIssuesOf("PWD-410")'
title = "Pilotbase Milestone 2 Scope"
days_to_extrapolate = 14
start_date="2024-09-01"
end_date="2024-09-30"

# query = 'issuekey in portfolioChildIssuesOf("FSP-7973")'
# title = "Student Financing with Stratus"
# days_to_extrapolate = 14
# start_date="2024-08-01"
# end_date="2024-12-01"

# query = 'issuekey in portfolioChildIssuesOf("FSP-7973")'
# title = "Stratus MVP Integration Scope"
# days_to_extrapolate = 14
# start_date="2024-06-15"
# end_date="2024-09-01"



# SETUP CONNECTIONS AND CONFIGURATION

In [ ]:
# Dependency install and import -- do both pip and pip3 to avoid issues.
%reload_ext autoreload
%autoreload 2
    
!pip -q install jira
!pip3 -q install jira
!pip -q install tqdm
!pip3 -q install tqdm
!pip -q install matplotlib
!pip3 -q install matplotlib
!pip -q install pandas
!pip3 -q install pandas

from jira import JIRA
import pandas as pd
import datetime
from datetime import *
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import math

In [ ]:
# Jira Connection
from jira_fsp_extracts import jira_connect

j = None if 'j' not in globals() else j
j = jira_connect(prompt_for_reconnect = jira_reconnect, existing_connection = j)

In [ ]:
# REFERENCE VALUES
from jira_references import *

# SUPPORT FUNCTIONS

In [ ]:
# def generate_burndown_chart() # GENERATE A BURNDOWN CHART WITH TOTAL SCOPE, RESOLVED SCOPE, REMAINING SCOPE
def generate_burndown_chart(df, start_date=None, end_date=None, extrapolate_days=None, title=None):
    """
    Generate a burndown chart from a DataFrame containing Jira issue data.

    Parameters:
    df (pd.DataFrame): DataFrame containing columns 'Created Date', 'Resolution Date', and 'Status'.

    Returns:
    None: Displays the burndown chart.
    """

    # Set the start and end date for the burndown chart
    if start_date is None:
        start_date = df['Created Date'].min()
    else: 
        start_date = pd.to_datetime(start_date).tz_localize(None)

    if end_date is None:
        end_date = df['Resolution Date'].max()
    else: 
        end_date = pd.to_datetime(end_date).tz_localize(None)
  
    # Ensure both start_date and end_date are timezone-naive (or have the same timezone)
    if start_date.tzinfo is not None:
        start_date = start_date.tz_convert(None)
    if end_date.tzinfo is not None:
        end_date = end_date.tz_convert(None)
        
    now = pd.Timestamp('now').normalize()
    
    if title is None:
        title = 'Burndown Chart'
        
    # Create a date range
    date_range = pd.date_range(start=start_date, end=end_date)

    # Initialize a DataFrame to hold the burndown data
    burndown_data = pd.DataFrame(date_range, columns=['Date'])
    burndown_data['Total Tasks'] = 0
    burndown_data['Remaining Tasks'] = 0
    burndown_data['Resolved Tasks'] = 0

    # Initialize extrapolation values
    daily_resolution_rate = None
    daily_creation_rate = None

    # Calculate the number of tasks remaining on each date
    for i, row in burndown_data.iterrows():
        date = row['Date']
        
        if date <= now:
            # prior to today we have actuals
            total_tasks = df[(df['Created Date'] <= date)].shape[0]
            resolved_tasks = df[(df['Resolution Date'] <= date)].shape[0]
            remaining_tasks = total_tasks - resolved_tasks
            
            burndown_data.at[i, 'Total Tasks'] = total_tasks
            burndown_data.at[i, 'Remaining Tasks'] = remaining_tasks
            burndown_data.at[i, 'Resolved Tasks'] = resolved_tasks
            burndown_data.at[i, 'Total Tasks (F)'] = None
            burndown_data.at[i, 'Remaining Tasks (F)'] = None
            burndown_data.at[i, 'Resolved Tasks (F)'] = None
        else:
            # Extrapolate future data if extrapolate_days is provided
            burndown_data.at[i, 'Total Tasks'] = None
            burndown_data.at[i, 'Remaining Tasks'] = None
            burndown_data.at[i, 'Resolved Tasks'] = None
            burndown_data.at[i, 'Total Tasks (F)'] = None
            burndown_data.at[i, 'Remaining Tasks (F)'] = None
            burndown_data.at[i, 'Resolved Tasks (F)'] = None
            
            if extrapolate_days is not None:
                # Calculate the trajectories the first time
                if daily_resolution_rate is None or daily_creation_rate is None:           
                    # Calculate the trajectory based on the last "X" days
                    trajectory_period = min(extrapolate_days, len(date_range))
                    recent_data = burndown_data[(burndown_data['Date'] <= now)].tail(trajectory_period)
                    
                    daily_resolution_rate = recent_data['Resolved Tasks'].diff().mean()
                    daily_creation_rate = recent_data['Total Tasks'].diff().mean()
                    print(f"Extrapolated outcomes based on {trajectory_period} days")
                    print(f"-- Daily Resolution Rate: {round(daily_resolution_rate,2)}")
                    print(f"-- Daily Creation Rate: {round(daily_creation_rate,2)}")
                    
                    burndown_data.at[i-1, 'Total Tasks (F)'] = burndown_data.at[i-1, 'Total Tasks']
                    burndown_data.at[i-1, 'Remaining Tasks (F)'] = burndown_data.at[i-1, 'Remaining Tasks']
                    burndown_data.at[i-1, 'Resolved Tasks (F)'] = burndown_data.at[i-1, 'Resolved Tasks']

                        
                burndown_data.at[i, 'Total Tasks (F)'] = burndown_data.at[i-1, 'Total Tasks (F)'] + daily_creation_rate
                burndown_data.at[i, 'Remaining Tasks (F)'] = burndown_data.at[i-1, 'Remaining Tasks (F)'] + daily_creation_rate - daily_resolution_rate
                burndown_data.at[i, 'Resolved Tasks (F)'] = burndown_data.at[i-1, 'Resolved Tasks (F)'] + daily_resolution_rate
            


    # Plot the burndown chart
    plt.figure(figsize=(10, 4))
    plt.plot(burndown_data['Date'], burndown_data['Total Tasks'], color='red', linestyle='-', label='Total Tasks')
    plt.plot(burndown_data['Date'], burndown_data['Remaining Tasks'], color='orange', linestyle='-', label='Remaining Tasks')
    plt.plot(burndown_data['Date'], burndown_data['Resolved Tasks'], color='green', linestyle='-', label='Resolved Tasks')

    if extrapolate_days is not None and (daily_resolution_rate is not None or daily_creation_rate is not None):
        if not burndown_data['Total Tasks (F)'].isna().all:
            plt.plot(burndown_data['Date'], burndown_data['Total Tasks (F)'], color='red', linestyle='dashed')
        if not burndown_data['Remaining Tasks (F)'].isna().all:
            plt.plot(burndown_data['Date'], burndown_data['Remaining Tasks (F)'], color='orange', linestyle='dashed')
        if not burndown_data['Resolved Tasks (F)'].isna().all:
            plt.plot(burndown_data['Date'], burndown_data['Resolved Tasks (F)'], color='green', linestyle='dashed')

    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Tickets')
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    #plt.tight_layout()
    plt.show()

In [ ]:
# def def generate_stacked_bar_chart() # GENERATE A CHART WITH TOTAL SCOPE BROKEN DOWN BY STATUS
def generate_stacked_bar_chart(df, start_date=None, end_date=None, extrapolate_days=None, title=None):
    """
    Generate a stacked bar chart based on statuses over time.

    Parameters:
    df (pd.DataFrame): DataFrame containing columns 'Created Date', 'Resolution Date', and 'Status'.

    Returns:
    None: Displays the burndown chart.
    """

    # Set the start and end date for the burndown chart
    if start_date is None:
        start_date = df['Created Date'].min()
    else: 
        start_date = pd.to_datetime(start_date).tz_localize(None)

    if end_date is None:
        end_date = df['Resolution Date'].max()
    else: 
        end_date = pd.to_datetime(end_date).tz_localize(None)
  
    # Ensure both start_date and end_date are timezone-naive (or have the same timezone)
    if start_date.tzinfo is not None:
        start_date = start_date.tz_convert(None)
    if end_date.tzinfo is not None:
        end_date = end_date.tz_convert(None)
        
    now = pd.Timestamp('now').normalize() + pd.Timedelta(days=1)
    
    if title is None:
        title = 'Stacked Bar Status Chart'
        
    # Create a date range
    date_range = pd.date_range(start=start_date, end=end_date)

    # Initialize a DataFrame to hold the burndown data
    burndown_data = pd.DataFrame(date_range, columns=['Date'])
    burndown_data['Total Tasks'] = 0
    burndown_data['PM Backlog Tasks'] = 0
    burndown_data['Eng Backlog Tasks'] = 0
    burndown_data['In Development Tasks'] = 0
    burndown_data['In Validation Tasks'] = 0
    burndown_data['Done Tasks'] = 0

    # Initialize extrapolation values
    daily_resolution_rate = None
    daily_creation_rate = None

    # Calculate the number of tasks remaining on each date
    for i, row in burndown_data.iterrows():
        date = row['Date']
        
        if date <= now:
            # prior to today we have actuals
            total_tasks = df[(df['Created Date'] <= date)].shape[0]
            done_tasks = df[(df['Done Date'] <= date)].shape[0] 
            in_validation_tasks = df[(df['Validation Date'] <= date) & ((df['Done Date'].isna()) | (df['Done Date'] > date))].shape[0] 
            in_development_tasks = df[(df['Development Date'] <= date) & ((df['Validation Date'].isna()) | (df['Validation Date'] > date)) & ((df['Done Date'].isna()) | (df['Done Date'] > date))].shape[0] 
            eng_backlog_tasks = df[(df['Eng Backlog Date'] <= date) & ((df['Development Date'].isna()) | (df['Development Date'] > date)) & ((df['Validation Date'].isna()) | (df['Validation Date'] > date)) & ((df['Done Date'].isna()) | (df['Done Date'] > date))].shape[0] 
            pm_backlog_tasks = df[(df['PM Backlog Date'] <= date) & ((df['Eng Backlog Date'].isna()) | (df['Eng Backlog Date'] > date)) & ((df['Development Date'].isna()) | (df['Development Date'] > date)) & ((df['Validation Date'].isna()) | (df['Validation Date'] > date)) & ((df['Done Date'].isna()) | (df['Done Date'] > date))].shape[0] 
            #pm_backlog_tasks = total_tasks - done_tasks - in_validation_tasks - in_development_tasks - eng_backlog_tasks
            
            burndown_data.at[i, 'Total Tasks'] = total_tasks
            burndown_data.at[i, 'PM Backlog Tasks'] = pm_backlog_tasks
            burndown_data.at[i, 'Eng Backlog Tasks'] = eng_backlog_tasks
            burndown_data.at[i, 'In Development Tasks'] = in_development_tasks
            burndown_data.at[i, 'In Validation Tasks'] = in_validation_tasks
            burndown_data.at[i, 'Done Tasks'] = done_tasks
            burndown_data.at[i, 'Total Tasks (F)'] = None
            burndown_data.at[i, 'Done Tasks (F)'] = None
            burndown_data.at[i, 'Remaining Tasks (F)'] = None
        else:
            # Extrapolate future data if extrapolate_days is provided
            burndown_data.at[i, 'Total Tasks'] = None
            burndown_data.at[i, 'PM Backlog Tasks'] = None
            burndown_data.at[i, 'Eng Backlog Tasks'] = None
            burndown_data.at[i, 'In Development Tasks'] = None
            burndown_data.at[i, 'In Validation Tasks'] = None
            burndown_data.at[i, 'Done Tasks'] = None
            burndown_data.at[i, 'Total Tasks (F)'] = None
            burndown_data.at[i, 'Done Tasks (F)'] = None
            burndown_data.at[i, 'Remaining Tasks (F)'] = None
            
            if extrapolate_days is not None:
                # Calculate the trajectories the first time
                if daily_resolution_rate is None or daily_creation_rate is None:           
                    # Calculate the trajectory based on the last "X" days
                    trajectory_period = min(extrapolate_days, len(date_range))
                    recent_data = burndown_data[(burndown_data['Date'] <= now)].tail(trajectory_period)
                    
                    daily_resolution_rate = recent_data['Done Tasks'].diff().mean()
                    daily_creation_rate = recent_data['Total Tasks'].diff().mean()
                    print(f"Extrapolated outcomes based on {trajectory_period} days")
                    print(f"-- Daily Resolution Rate: {round(daily_resolution_rate,2)}")
                    print(f"-- Daily Creation Rate: {round(daily_creation_rate,2)}")
                    
                    burndown_data.at[i-1, 'Total Tasks (F)'] = burndown_data.at[i-1, 'Total Tasks']
                    burndown_data.at[i-1, 'Done Tasks (F)'] = burndown_data.at[i-1, 'Done Tasks']
                    burndown_data.at[i-1, 'Remaining Tasks (F)'] = burndown_data.at[i-1, 'Total Tasks (F)'] - burndown_data.at[i-1, 'Done Tasks (F)']
            
                burndown_data.at[i, 'Total Tasks (F)'] = burndown_data.at[i-1, 'Total Tasks (F)'] + daily_creation_rate
                burndown_data.at[i, 'Done Tasks (F)'] = burndown_data.at[i-1, 'Done Tasks (F)'] + daily_resolution_rate
                if burndown_data.at[i, 'Done Tasks (F)'] < 0:
                    burndown_data.at[i, 'Done Tasks (F)'] = 0
                burndown_data.at[i, 'Remaining Tasks (F)'] = burndown_data.at[i, 'Total Tasks (F)'] - burndown_data.at[i, 'Done Tasks (F)']

    # For troubleshooting
    burndown_data.to_csv("burndown_data.csv")
    
    # Plot the burndown chart
    fig, ax = plt.subplots(figsize=(10, 4))
    
    ax.plot(burndown_data['Date'], burndown_data['Total Tasks'], color='black', linestyle='-', label='Total')

    # Plot the stacked bars
    ax.bar(burndown_data['Date'], burndown_data['PM Backlog Tasks'], label='PM Backlog', color='red')
    ax.bar(burndown_data['Date'], burndown_data['Eng Backlog Tasks'], bottom=burndown_data['PM Backlog Tasks'], label='Eng Backlog', color='orange')
    ax.bar(burndown_data['Date'], burndown_data['In Development Tasks'], bottom=burndown_data['PM Backlog Tasks']+burndown_data['Eng Backlog Tasks'], label='In Development', color='yellow')
    ax.bar(burndown_data['Date'], burndown_data['In Validation Tasks'], bottom=burndown_data['PM Backlog Tasks']+burndown_data['Eng Backlog Tasks']+burndown_data['In Development Tasks'], label='In Validation', color='green')
    ax.bar(burndown_data['Date'], burndown_data['Done Tasks'], bottom=burndown_data['PM Backlog Tasks']+burndown_data['Eng Backlog Tasks']+burndown_data['In Development Tasks']+burndown_data['In Validation Tasks'], label='Done', color='blue')

    if extrapolate_days is not None and (daily_resolution_rate is not None or daily_creation_rate is not None):
        forecasted_data = burndown_data[burndown_data['Done Tasks (F)'].notna()]
        if not forecasted_data['Total Tasks (F)'].isna().all():
            plt.plot(forecasted_data['Date'], forecasted_data['Total Tasks (F)'], color='black', linestyle='dashed')
        if not forecasted_data['Done Tasks (F)'].isna().all():
            ax.bar(forecasted_data['Date'], forecasted_data['Done Tasks (F)'], bottom=forecasted_data['Remaining Tasks (F)'], color='blue', alpha=0.4)
        
    plt.title(title)
    plt.xlabel('Date')
    plt.ylabel('Tickets')

    y_limit = math.ceil(max(burndown_data['Total Tasks'].max(), burndown_data['Total Tasks (F)'].max()) + 2)
    plt.ylim(0, y_limit)
    plt.grid(True)
    plt.legend()
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [ ]:
# def generate_bar_chart() # GENERATE A BAR CHART BASED ON VOLUME FOR AN IDENTIFIED COLUMN
def generate_bar_chart(df, column, title=None):
    """
    GENERATE A BAR CHART BASED ON VOLUME FOR AN IDENTIFIED COLUMN

    Parameters:
    df (pd.DataFrame): DataFrame containing columns 'Created Date', 'Resolution Date', and 'Status'.
    column (string): Name of the column for the buckets used on the horizontal axis.

    Returns:
    None: Displays the chart.
    """
    
    if title is None:
        title = 'Bar Chart'
    
    bin_counts = df[column].value_counts().sort_index()

    plt.figure(figsize=(10, 6))
    bin_counts.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title(title)
    plt.xlabel(column)
    plt.ylabel('Number')
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

In [ ]:
# def generate_whisker_chart() # GENERATE A WHISKER CHART BASED ON VOLUME FOR AN IDENTIFIED COLUMN
def generate_whisker_chart(df, columns, ylabel=None, title=None):
    """
    GENERATE A WHISKER CHART BASED ON SELECTED COLUMNS

    Parameters:
    df (pd.DataFrame): DataFrame 
    columns (string[]): Name of the columns for analysis.

    Returns:
    None: Displays the chart.
    """
    
    if title is None:
        title = 'Bar Chart'
        
    # Create the box plot
    # Define colors for the box plots
    colors = {
        'boxes': 'DarkOrange',
        'whiskers': 'DarkOrange',
        'medians': 'Blue',
        'caps': 'Green'
    }
    
    plt.figure(figsize=(8, 6))
    try:
        boxplot = df[columns].boxplot(
            patch_artist=True,
            boxprops=dict(facecolor='lightblue', color='DarkOrange'),
            capprops=dict(color=colors['caps']),
            whiskerprops=dict(color=colors['whiskers']),
            flierprops=dict(markerfacecolor='red', marker='o', markersize=5, linestyle='none'),
            medianprops=dict(color=colors['medians']),
            vert=True
        )
    except ValueError as e:
        print("ValueError encountered: ", e)
        print("Ensure that the DataFrame contains data to generate a plot for the specified columns.")
        # Optionally, you can return or raise an error or handle it differently
        
    plt.title(title)
    if ylabel:
        plt.ylabel(ylabel)
        
    plt.xticks(rotation=45)

# COLLECT AND ANALYZE DATA FOR REPORTS

In [ ]:
# RETRIEVE DATA
from jira_fsp_extracts import fetch_jira_issues_to_dataframe

final_query = '(' + query + ') and ((resolution is empty or resolution = Done) and status != "Won\'t Do")'
print(f"JQL Query: {final_query}")

df = fetch_jira_issues_to_dataframe(jira_conn = j, jql_query = final_query)

In [ ]:
# Cycle time calculations
# Defintions:
# Lead Time: Created until Done
    # Cycle Time: Development Start until Done
        # Development Time: Development Start until QA Review
        # Validation Time: QA Start Until Done
# + Release: Base Metric + Time from Done to Release (FUTURE)

# only do calcs on done tickets
df_ct_calcs = df[df['Done Date'].notna()].copy()
if df_ct_calcs.shape[0] > 0:
    df_ct_calcs["Lead Time (days)"] = round((df_ct_calcs['Done Date'] - df_ct_calcs['Created Date']).dt.total_seconds() / (24 * 60 * 60),2)
    df_ct_calcs["Cycle Time (days)"] = round((df_ct_calcs['Done Date'] - df_ct_calcs['Development Date']).dt.total_seconds() / (24 * 60 * 60),2)
    df_ct_calcs["CT Development Time (days)"] = round((df_ct_calcs['Validation Date'] - df_ct_calcs['Development Date']).dt.total_seconds() / (24 * 60 * 60),2)
    df_ct_calcs["CT Validation Time (days)"] = round((df_ct_calcs['Done Date'] - df_ct_calcs['Validation Date']).dt.total_seconds() / (24 * 60 * 60),2)

    # Calculate 'Release Time (days)' only when 'Release Date' is not None
    df_ct_calcs["Release Time (days)"] = df_ct_calcs.apply(
        lambda row: round((row['Release Date'] - row['Done Date']).total_seconds() / (24 * 60 * 60), 2) if row['Release Date'] is not None else None,
        axis=1)
else:
    df_ct_calcs["Lead Time (days)"] = None
    df_ct_calcs["Cycle Time (days)"] = None
    df_ct_calcs["CT Development Time (days)"] = None
    df_ct_calcs["CT Validation Time (days)"] = None
    df_ct_calcs["Release Time (days)"] = None
    
# merge calced data into primary df
# Drop the column if it exists
df = df.drop(columns=["Lead Time (days)","Cycle Time (days)","CT Development Time (days)","CT Validation Time (days)","Release Time (days)"], errors='ignore')
df = df.merge(df_ct_calcs[['Issue Key',"Lead Time (days)","Cycle Time (days)","CT Development Time (days)","CT Validation Time (days)","Release Time (days)"]],on="Issue Key",how="left")

In [ ]:
# Create subsets for future reports
df_filtered_standard_issues = df[df['Issue Type Category'] == "STANDARD"]
df_filtered_sub_tasks_issues = df[df['Issue Type Category'] == "SUBTASK"]

In [ ]:
# Analyze Defects / other subtasks
defects = df_filtered_sub_tasks_issues[df_filtered_sub_tasks_issues['Issue Type'] == "Defect"].groupby(['Parent Story']).agg({'Issue Key': 'count'}).rename(columns={'Issue Key': 'Defect Count'})
sub_tasks_analysis = df_filtered_sub_tasks_issues.groupby(['Parent Story']).agg({'Issue Key': 'count'}).rename(columns={'Issue Key': 'All Subtasks Count'})
sub_tasks_analysis = sub_tasks_analysis.merge(defects, on="Parent Story", how="left").fillna(0)
sub_tasks_analysis['Defect Percentage'] = (sub_tasks_analysis['Defect Count'] / sub_tasks_analysis['All Subtasks Count']) * 100

# Bucket the percentages
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-10%', '10-20%', '20-30%', '30-40%', '40-50%', '50-60%', '60-70%', '70-80%', '80-90%', '90-100%']
sub_tasks_analysis['Defect Percentage Bin'] = pd.cut(sub_tasks_analysis['Defect Percentage'], bins=bins, labels=labels, include_lowest=True)


# BURNDOWN STACKED BAR

In [ ]:
generate_stacked_bar_chart(df_filtered_standard_issues, start_date=start_date, end_date=end_date, extrapolate_days=days_to_extrapolate, title=title + ' - Story Burndown (Remaining scope should be 0 by launch date)')
generate_stacked_bar_chart(df_filtered_sub_tasks_issues, start_date=start_date, end_date=end_date, extrapolate_days=days_to_extrapolate, title=title + ' - Sub-Task Burndown (Remaining scope should be stable in a flow state)')
generate_stacked_bar_chart(df, start_date=start_date, end_date=end_date, extrapolate_days=days_to_extrapolate, title=title + ' - All Burndown (Remaining scope should be 0 by launch date)')

# DEFECTS PER STORY

In [ ]:
generate_bar_chart(sub_tasks_analysis,'Defect Percentage Bin',"% of Defect Subtasks")

# CYCLE TIME

In [ ]:
generate_whisker_chart(df_filtered_standard_issues, 
                       columns=['Lead Time (days)', 'Cycle Time (days)', 'CT Development Time (days)', 'CT Validation Time (days)', 'Release Time (days)'], 
                       ylabel="Days", title="Cycle Time Report - Stories")
generate_whisker_chart(df_filtered_sub_tasks_issues, 
                       columns=['Lead Time (days)', 'Cycle Time (days)', 'CT Development Time (days)', 'CT Validation Time (days)', 'Release Time (days)'], 
                       ylabel="Days", title="Cycle Time Report - SubTasks")